In [1]:
%load_ext autoreload
%autoreload 2

Clone the repo and install stuffs

In [3]:
!git clone https://github.com/longxvu/Adaptive_CA

Cloning into 'Adaptive_CA'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0
Receiving objects: 100% (9/9), 18.81 KiB | 18.81 MiB/s, done.


In [4]:
from google.colab import files

files.os.chdir("Adaptive_CA")

# To find local version of the library
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))

Adaptive_CA  sample_data


In [2]:
from assistant import GPTAssistant
from utils import show_json, list_all_assistant, get_api_key
from tool_functions import display_quiz_function_json
from openai import OpenAI
import pandas as pd
import json

ModuleNotFoundError: No module named 'assistant'

## Init client and choosing our previously created assistant

In [ ]:
# Create client and retrieve all current assistant
client = OpenAI(
    api_key=get_api_key()
)   # Maybe client should be outside assistant class but this will do for now

ipykernel.zmqshell


In [ ]:
list_all_assistant(client)

id: asst_ZNz4lbi6z8bpKkE6APKdrpQ8
name: None
description: Assistant on helping children learn science concepts
instructions: You will be presented with a couple of dialogue between two cartoon characters.You need to identify a few science concepts presented in the dialogue to assist the children in learning the concepts.
model: gpt-3.5-turbo-0125
tools: [{'function': {'name': 'display_quiz', 'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.", 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string'}, 'questions': {'type': 'array', 'description': 'An array of questions, each with a title and potentially options (if multiple choice).', 'items': {'type': 'object', 'properties': {'question_text': {'type': 'string'}, 'question_type': {'type': 'string', 'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']}, 'difficulty': {'type': 'string', 'description': 'The difficulty level of each question. Easy difficu

In [ ]:
# update the function that assistant will use
# client.beta.assistants.update(assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8", tools=[{"type": "function", "function": display_quiz_function_json}])
# client.beta.assistants.update(assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8", instructions="You will be presented with a couple of dialogue between two cartoon characters."
#                                                                                           "You need to identify a few science concepts presented in the dialogue to assist the children in learning "
#                                                                                           "the concepts.")

In [ ]:
assistant = GPTAssistant(client, assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8")

In [ ]:
print(assistant.assistant.id)
print(assistant.thread.id)
print(assistant.get_all_messages())

asst_ZNz4lbi6z8bpKkE6APKdrpQ8
thread_GV91GiJ1fSKMxwRfuxugJDL4
[]


## Get dialogue data from xlsx file

In [ ]:
dialogue_file_path = "transcripts/town_picnic_base.xlsx"
df = pd.read_excel(dialogue_file_path)

In [ ]:
transcripts = df[df["text"].notnull()]["text"].tolist()
base_question = df[df["text"].notnull()]["base_q"].tolist()

In [ ]:
print(transcripts[0][0:500] + "...")
print("Base question:", base_question[0])

Elinor: The Town Picnic! [Elinor hops into view from the center. She giggles as she notices a bee and runs after it to a flower] Hmm? Oooh...
Ranger: [off-screen] C'mon, Elinor!
Elinor: Coming, Mom! [Elinor runs to the left, where Ranger follows behind her. Elinor runs to the entrance of the picnic grounds.] I just love the Town Picnic! What should we do first, Mom?
Ranger: I don't know, Elinor. Let's see... [gasp] Oh, we can do wood-carving with Mrs. Beaver. [Cut to Mrs. Beaver rapidly biting s...
Base question: Did you see that!?- the bee just drank some juice, called nectar from this flower! What do you think the bee is going to make with the juice from this flower?


## Main GPT dialogue loop

In [ ]:
# Can set it to run forever but currently restricting to a limited amount of step for testing
max_step = 1
# num_questions = 3
step = 0
cur_dialogue_num = 0

In [ ]:
# assistant.wait_on_run()
# tool_calls = assistant.last_run.required_action.submit_tool_outputs.tool_calls
# for tool_call in tool_calls:
#     print(json.loads(tool_call.function.arguments))

In [ ]:
# assistant.resolve_run_required_action()
# assistant.get_all_messages()

First, the assistant is presented with the dialogue then it is asked to summarize it. Then we ask it to create a quiz of varying difficulties about the science concept presented in the dialogue

In [ ]:
while step < max_step and cur_dialogue_num < len(transcripts):
    # Two steps: First, summarize the dialogue (for easier testing, can be omitted). Then ask the question about summarized dialogue
    assistant.submit_message(f"Summarize this dialogue: {transcripts[cur_dialogue_num]}.")
    print(assistant.get_last_response())
    assistant.submit_message(f"""
    Create a quiz with 3 questions to ask the child about the science concept presented in the dialogue. There should be one easy, one medium, and one hard question.
    Then, based on their answer, identify what did the child get wrong and gave them feedback.
    """)
    assistant.wait_on_run()
    if assistant.run_requires_action():
        assistant.resolve_run_required_action()
    print(assistant.get_last_response())

    step += 1
    cur_dialogue_num += 1
    print("==============================================================================")

assistant: The dialogue is about Elinor and her family attending the Town Picnic. Elinor is excited about the activities at the picnic, such as wood-carving, hayrides, and watching Mr. Cat at the dunking booth. Later, Elinor joins her friends Olive and Ari in line for Mrs. Llama's special ketchup, where they express impatience due to the long wait. Elinor gets distracted by a bee and follows it to a flower, where she observes the bee sucking up something from the flower.

Concepts identified:
1. Pollination: Elinor follows a bee to a flower and observes the bee sucking up something from it, indicating a learning opportunity about pollination.
2. Waiting in line: Olive, Ari, and Elinor experience a long wait in line for food, showcasing the concept of patience and waiting in a queue.
3. Food preferences: Ari waits in line for Mrs. Llama's special ketchup, highlighting the concept of different food preferences and the anticipation of trying something new.
display_quiz
{'title': 'Science 

Answer:  a



Questions: Medium: What concept is showcased when Olive, Ari, and Elinor experience a long wait in line for food?


Answer:  Idk



Questions: Hard: How does the interaction between the bee and flower in the dialogue relate to the process of pollination?


Answer:  Bee delivers honey to flowers



['a', 'Idk', 'Bee delivers honey to flowers']
assistant: 1. The child got the easy question wrong. The correct answer to the question "What did Elinor follow to a flower in the dialogue?" is "A bee." Bees are important pollinators that transfer pollen between flowers.
2. The child did not provide an answer for the medium question. The concept showcased when Olive, Ari, and Elinor experience a long wait in line for food is patience and waiting in queue.
3. The child misunderstood the hard question. The interaction between the bee and flower in the dialogue relates to the process of pollination where bees visit flowers to collect nectar and pollen, aiding in the reproduction of plants. The bee transferring pollen from one flower to another is a crucial step in pollination.


In [ ]:
# for msg in assistant.get_all_messages():
#     print(msg)

user: Summarize this dialogue: Elinor: The Town Picnic! [Elinor hops into view from the center. She giggles as she notices a bee and runs after it to a flower] Hmm? Oooh...
Ranger: [off-screen] C'mon, Elinor!
Elinor: Coming, Mom! [Elinor runs to the left, where Ranger follows behind her. Elinor runs to the entrance of the picnic grounds.] I just love the Town Picnic! What should we do first, Mom?
Ranger: I don't know, Elinor. Let's see... [gasp] Oh, we can do wood-carving with Mrs. Beaver. [Cut to Mrs. Beaver rapidly biting some wood into an intricate carving while saw sounds play. Cut back to Elinor.]
Elinor: Awesome!
Ranger: Or... Mr. Horse is giving everyone hayrides. [Cut to Mr. Horse struggling to pull a wagon with the Hippo family on it.]
Elinor: Oh yeah!
Ranger: Or... We could watch Mr. Cat at the dunking booth! [Cut to Mr. Cat sitting atop a dunking booth, who is shortly dunked.]
Elinor: [gasp] Woo-hoo!
Olive: [off-screen] Elinor!
Elinor: Huh? [Cut to a line of people holding s